In [1]:
library("coloc")
suppressMessages(suppressWarnings(library(tidyverse)))
library(locuscomparer)
library(susieR)
set.seed(1)
suppressMessages(library('Rfast'))

This is coloc version 5.1.1

Warning message:
"package 'Rfast' was built under R version 4.1.3"
Warning message:
"package 'Rcpp' was built under R version 4.1.3"
Warning message:
"package 'RcppZiggurat' was built under R version 4.1.3"


In [50]:
#metsim

In [20]:
df_gwas <- list.files("../../ANALYSIS/COLOC_SUSIE/harmonized_metsim/gwas/",pattern='.z',full.names = TRUE)
df_gwas <- df_gwas[ !grepl(".bgen.bcor.ld", df_gwas) ]
df_eqtl <- list.files("../../ANALYSIS/COLOC_SUSIE/harmonized_metsim/eqtl/",pattern='.z',full.names = TRUE)
df_eqtl <- df_eqtl[ !grepl(".bgen.bcor.ld", df_eqtl) ]

In [30]:
df_gwas[16]

[1] "../../ANALYSIS/COLOC_SUSIE/harmonized_metsim/gwas/Adipose_Visceral_Omentum_3-88485.846.z"

In [39]:
i=16
# tissue
tissue <- tail(str_split(df_gwas,"/|.z")[[i]],n=3)[2]
 # gwas
s_gwas <- read.table(df_gwas[i],header=T,sep=" ")
z_gwas=s_gwas$beta/s_gwas$se
R_gwas <- as.matrix(read.table(paste0(df_gwas[i],".bgen.bcor.ld"),header=FALSE)%>%replace(is.na(.), 0.001))
fitted_gwas <- susie_rss(z_gwas,R_gwas,estimate_prior_variance = FALSE,max_iter = 5000,L=10,coverage=0.90,n=10000)
# eqtl
s_eqtl <- read.table(df_eqtl[i],header=T,sep=" ")
z_eqtl=s_eqtl$beta/s_eqtl$se
R_eqtl <- as.matrix(read.table(paste0(df_eqtl[i],".bgen.bcor.ld"),header=FALSE)%>%replace(is.na(.), 0.001))
fitted_eqtl <- susie_rss(z_eqtl,R_eqtl,estimate_prior_variance = FALSE,max_iter = 5000,L=10,coverage=0.90,n=10000)
    
## coloc and susie
susie.res=coloc.susie(fitted_gwas,fitted_eqtl)

Warning message in susie_suff_stat(XtX = XtX, Xty = Xty, n = n, yty = (n - 1) * :
"IBSS algorithm did not converge in 5000 iterations!
                  Please check consistency between summary statistics and LD matrix.
                  See https://stephenslab.github.io/susieR/articles/susierss_diagnostic.html"


In [ ]:
##

In [16]:
pip_df <- data.frame()
suppressMessages(suppressWarnings(
for (i in seq_along(df_gwas))
{
tryCatch({
# tissue
tissue <- tail(str_split(df_gwas,"/|.z")[[i]],n=3)[2]
    # gwas
s_gwas <- read.table(df_gwas[i],header=T,sep=" ")
z_gwas=s_gwas$beta/s_gwas$se
R_gwas <- as.matrix(read.table(paste0(df_gwas[i],".bgen.bcor.ld"),header=FALSE)%>%replace(is.na(.), 0))
fitted_gwas <- susie_rss(z_gwas,R_gwas,estimate_prior_variance = FALSE,max_iter = 5000,L=10,coverage=0.90)
# eqtl
s_eqtl <- read.table(df_eqtl[i],header=T,sep=" ")
z_eqtl=s_eqtl$beta/s_eqtl$se
R_eqtl <- as.matrix(read.table(paste0(df_eqtl[i],".bgen.bcor.ld"),header=FALSE)%>%replace(is.na(.), 0))
fitted_eqtl <- susie_rss(z_eqtl,R_eqtl,estimate_prior_variance = FALSE,max_iter = 5000,L=10,coverage=0.90)
    
## coloc and susie
susie.res=coloc.susie(fitted_gwas,fitted_eqtl)
}, error=function(e){})
## data frame
if (length(s_gwas$rsid)==length(susie.res$results$SNP.PP.H4.row4)){

df <- data.frame(MARKERNAME=s_gwas$rsid,H3_abf=susie.res$results$SNP.PP.H4.row3,H4_abf=susie.res$results$SNP.PP.H4.row4,eqtl_beta=s_eqtl$beta,eqtl_se=s_eqtl$se)%>%mutate(Tissue=tissue)
pip_df   <- rbind(pip_df ,df)
    }
    }
))

In [19]:
head(pip_df)

<0 x 0 matrix>

In [ ]:
out_df_H4_temp <- merge(TRANSANCESTRY_RESULTS,pip_df,by="MARKERNAME")%>%select(MARKERNAME,CHROMOSOME,POSITION,METSIM_EA,METSIM_NEA,PVALUE_FE,ADIPO_EAF,AGEN_EAF,METSIM_EAF,TOTAL_N,eqtl_beta,eqtl_se,H4_abf,Tissue)%>%
filter(H4_abf>=0 )

In [2]:
## trans

In [2]:
df_gwas <- list.files("../../ANALYSIS/COLOC_SUSIE/harmonized_trans/gwas/",pattern='.z',full.names = TRUE)
df_gwas <- df_gwas[ !grepl(".bgen.bcor.ld", df_gwas) ]
df_eqtl <- list.files("../../ANALYSIS/COLOC_SUSIE/harmonized_trans/eqtl/",pattern='.z',full.names = TRUE)
df_eqtl <- df_eqtl[ !grepl(".bgen.bcor.ld", df_eqtl) ]

In [3]:
### check if files are ok

In [4]:
for (i in seq_along(df_gwas))
{
s_gwas <- nrow(read.table(df_gwas[i],header=T,sep=" "))
R_gwas <- nrow(as.matrix(read.table(paste0(df_gwas[i],".bgen.bcor.ld"),header=FALSE)))
s_eqtl <- nrow(read.table(df_eqtl[i],header=T,sep=" "))
R_eqtl <- nrow(as.matrix(read.table(paste0(df_eqtl[i],".bgen.bcor.ld"),header=FALSE)))

if (s_gwas!=s_eqtl & R_gwas!=R_eqtl){
 print(df_gwas[i])   
}    
}

In [5]:
## coloc and susie

In [50]:
pip_df <- data.frame()
suppressMessages(suppressWarnings(
for (i in seq_along(df_gwas))
{
tryCatch({
# tissue
tissue <- tail(str_split(df_gwas,"/|.z")[[i]],n=3)[2]
    # gwas
s_gwas <- read.table(df_gwas[i],header=T,sep=" ")
z_gwas=s_gwas$beta/s_gwas$se
R_gwas <- as.matrix(read.table(paste0(df_gwas[i],".bgen.bcor.ld"),header=FALSE))
fitted_gwas <- susie_rss(z_gwas,R_gwas,estimate_prior_variance = FALSE,max_iter = 5000,L=10,coverage=0.90)
# eqtl
s_eqtl <- read.table(df_eqtl[i],header=T,sep=" ")
z_eqtl=s_eqtl$beta/s_eqtl$se
R_eqtl <- as.matrix(read.table(paste0(df_eqtl[i],".bgen.bcor.ld"),header=FALSE))
fitted_eqtl <- susie_rss(z_eqtl,R_eqtl,estimate_prior_variance = FALSE,max_iter = 5000,L=10,coverage=0.90)
    
## coloc and susie
susie.res=coloc.susie(fitted_gwas,fitted_eqtl)
}, error=function(e){})
## data frame
if (length(s_gwas$rsid)==length(susie.res$results$SNP.PP.H4.row4)){

df <- data.frame(MARKERNAME=s_gwas$rsid,H3_abf=susie.res$results$SNP.PP.H4.row3,H4_abf=susie.res$results$SNP.PP.H4.row4,eqtl_beta=s_eqtl$beta,eqtl_se=s_eqtl$se)%>%mutate(Tissue=tissue)
pip_df   <- rbind(pip_df ,df)
    }
}
))

PP.H0.abf PP.H1.abf PP.H2.abf PP.H3.abf PP.H4.abf 
 0.00e+00  1.98e-17  0.00e+00  1.00e+00  1.03e-11 
[1] "PP abf for shared variant: 1.03e-09%"
PP.H0.abf PP.H1.abf PP.H2.abf PP.H3.abf PP.H4.abf 
 0.00e+00  1.98e-17  0.00e+00  1.00e+00  9.62e-12 
[1] "PP abf for shared variant: 9.62e-10%"
PP.H0.abf PP.H1.abf PP.H2.abf PP.H3.abf PP.H4.abf 
 0.00e+00  9.09e-07  0.00e+00  1.00e+00  6.64e-09 
[1] "PP abf for shared variant: 6.64e-07%"
PP.H0.abf PP.H1.abf PP.H2.abf PP.H3.abf PP.H4.abf 
 0.00e+00  9.09e-07  0.00e+00  1.00e+00  6.66e-09 
[1] "PP abf for shared variant: 6.66e-07%"
PP.H0.abf PP.H1.abf PP.H2.abf PP.H3.abf PP.H4.abf 
 0.00e+00  6.16e-09  0.00e+00  1.00e+00  9.98e-11 
[1] "PP abf for shared variant: 9.98e-09%"
PP.H0.abf PP.H1.abf PP.H2.abf PP.H3.abf PP.H4.abf 
 0.00e+00  6.16e-09  0.00e+00  1.00e+00  9.98e-11 
[1] "PP abf for shared variant: 9.98e-09%"
PP.H0.abf PP.H1.abf PP.H2.abf PP.H3.abf PP.H4.abf 
 0.00e+00  2.86e-07  0.00e+00  3.98e-03  9.96e-01 
[1] "PP abf for shared varia

ERROR: Error in rbind(deparse.level, ...): numbers of columns of arguments do not match


In [7]:
Metasoft_results <-  read.table("../../ANALYSIS/META/METASOFT/SECOND_PASS/TRANSANCESTRY_METASOFT_SECONDPASS_RESULTS.TAB",fill=TRUE,row.names=NULL,header=FALSE)
META_THREE <- read.table(gzfile("../data/TRANSANCESTRY_GWAS_ALLELECORRECTED.TAB"),header=T,fill=TRUE,row.names=NULL)
colnames(Metasoft_results) <- c("MARKERNAME","N","PVALUE_FE","BETA_FE","STD_FE","PVALUE_RE","BETA_RE","STD_RE","PVALUE_RE2","STAT1_RE2","STAT2_RE2","PVALUE_BE","I_SQUARE","Q","PVALUE_Q","TAU_SQUARE",
                                "PVALUE_ADIPOGEN","PVALUE_AGEN","PVALUE_METSIM","NA1","NA2","NA3")
Metasoft_results <- Metasoft_results%>%filter(MARKERNAME!="RSID")
TRANSANCESTRY_RESULTS <- merge(META_THREE,Metasoft_results,by="MARKERNAME")
TRANSANCESTRY_RESULTS <- TRANSANCESTRY_RESULTS%>%mutate(TOTAL_N=ceiling(ADIPO_N)+ceiling(AGEN_N)+ceiling(METSIM_N))%>%mutate(MAF_AVG=(ADIPO_EAF*2*ADIPO_N+AGEN_EAF*2*AGEN_N+METSIM_EAF*2*METSIM_N)/(2*TOTAL_N))%>%
filter(ADIPO_EAF>0.005)%>%filter(AGEN_EAF>0.005)%>%filter(METSIM_EAF>0.005)%>%select(-NA1,-N,-NA2,-NA3)


In [8]:
df_expr <- list.files("../../ANALYSIS/COLOC_SUSIE/trans/",pattern='.txt',full.names = TRUE)
expr_df <- data.frame()
for (i in seq_along(df_expr))
{
tissue <- tail(str_split(df_expr,"/|.txt")[[i]],n=3)[1]
eq_df <- read.table(df_expr[i],header=T,sep="\t")%>%rename(MARKERNAME=SNP,eqtl_beta=b,eqtl_se=SE)%>%mutate(Tissue=tissue)
expr_df   <- rbind(expr_df ,eq_df)
}    

In [9]:
out_df_H4_temp <- merge(TRANSANCESTRY_RESULTS,pip_df,by="MARKERNAME")%>%select(MARKERNAME,CHROMOSOME,POSITION,METSIM_EA,METSIM_NEA,PVALUE_FE,ADIPO_EAF,AGEN_EAF,METSIM_EAF,TOTAL_N,eqtl_beta,eqtl_se,H4_abf,Tissue)%>%
filter(H4_abf>=0 )
out_df_H4 <- merge(out_df_H4_temp,expr_df,by=c("MARKERNAME","Tissue","eqtl_beta","eqtl_se"))
out_df_H3_temp <- merge(TRANSANCESTRY_RESULTS,pip_df,by="MARKERNAME")%>%select(MARKERNAME,CHROMOSOME,POSITION,METSIM_EA,METSIM_NEA,PVALUE_FE,ADIPO_EAF,AGEN_EAF,METSIM_EAF,TOTAL_N,eqtl_beta,eqtl_se,H3_abf,Tissue)%>%
filter(H3_abf>0.9 )
out_df_H3 <- merge(out_df_H3_temp,expr_df,by=c("MARKERNAME","Tissue","eqtl_beta","eqtl_se"))

In [10]:
write.table(out_df_H4,"../../ANALYSIS/COLOC_SUSIE/TRANS_ANCESTRY_COLOC_H4.TAB",row.names=FALSE,quote=FALSE,sep="\t")
write.table(out_df_H3,"../../ANALYSIS/COLOC_SUSIE/TRANS_ANCESTRY_COLOC_H3.TAB",row.names=FALSE,quote=FALSE,sep="\t")

In [17]:
### European

In [18]:
df_gwas <- list.files("../../ANALYSIS/COLOC_SUSIE/harmonized_eur/gwas/",pattern='.z',full.names = TRUE)
df_gwas <- df_gwas[ !grepl(".bgen.bcor.ld", df_gwas) ]
df_eqtl <- list.files("../../ANALYSIS/COLOC_SUSIE/harmonized_eur/eqtl/",pattern='.z',full.names = TRUE)
df_eqtl <- df_eqtl[ !grepl(".bgen.bcor.ld", df_eqtl) ]

In [19]:
##check

In [20]:
for (i in seq_along(df_gwas))
{
s_gwas <- nrow(read.table(df_gwas[i],header=T,sep=" "))
R_gwas <- nrow(as.matrix(read.table(paste0(df_gwas[i],".bgen.bcor.ld"),header=FALSE)))
s_eqtl <- nrow(read.table(df_eqtl[i],header=T,sep=" "))
R_eqtl <- nrow(as.matrix(read.table(paste0(df_eqtl[i],".bgen.bcor.ld"),header=FALSE)))

if (s_gwas!=s_eqtl & R_gwas!=R_eqtl){
 print(df_gwas[i])   
}    
}

In [21]:
##coloc and susie

In [22]:
pip_df <- data.frame()
suppressMessages(suppressWarnings(
for (i in seq_along(df_gwas))
{
tryCatch({
# tissue
tissue <- tail(str_split(df_gwas,"/|.z")[[i]],n=3)[2]
    # gwas
s_gwas <- read.table(df_gwas[i],header=T,sep=" ")
z_gwas=s_gwas$beta/s_gwas$se
R_gwas <- as.matrix(read.table(paste0(df_gwas[i],".bgen.bcor.ld"),header=FALSE))
fitted_gwas <- susie_rss(z_gwas,R_gwas,estimate_prior_variance = FALSE,max_iter = 5000,L=10,coverage=0.90)
# eqtl
s_eqtl <- read.table(df_eqtl[i],header=T,sep=" ")
z_eqtl=s_eqtl$beta/s_eqtl$se
R_eqtl <- as.matrix(read.table(paste0(df_eqtl[i],".bgen.bcor.ld"),header=FALSE))
fitted_eqtl <- susie_rss(z_eqtl,R_eqtl,estimate_prior_variance = FALSE,max_iter = 5000,L=10,coverage=0.90)
    
## coloc and susie
susie.res=coloc.susie(fitted_gwas,fitted_eqtl)
}, error=function(e){})
## data frame
if (length(s_gwas$rsid)==length(susie.res$results$SNP.PP.H4.row4)){

df <- data.frame(MARKERNAME=s_gwas$rsid,H3_abf=susie.res$results$SNP.PP.H4.row3,H4_abf=susie.res$results$SNP.PP.H4.row4,eqtl_beta=s_eqtl$beta,eqtl_se=s_eqtl$se)%>%mutate(Tissue=tissue)
pip_df   <- rbind(pip_df ,df)
    }
    }
))

PP.H0.abf PP.H1.abf PP.H2.abf PP.H3.abf PP.H4.abf 
 0.00e+00  0.00e+00  4.83e-20  1.00e+00  3.98e-05 
[1] "PP abf for shared variant: 0.00398%"
PP.H0.abf PP.H1.abf PP.H2.abf PP.H3.abf PP.H4.abf 
  0.00000   0.00000   0.64900   0.34600   0.00501 
[1] "PP abf for shared variant: 0.501%"
PP.H0.abf PP.H1.abf PP.H2.abf PP.H3.abf PP.H4.abf 
 0.00e+00  0.00e+00  4.83e-20  1.00e+00  8.50e-05 
[1] "PP abf for shared variant: 0.0085%"
PP.H0.abf PP.H1.abf PP.H2.abf PP.H3.abf PP.H4.abf 
  0.00000   0.00000   0.65000   0.34600   0.00486 
[1] "PP abf for shared variant: 0.486%"
PP.H0.abf PP.H1.abf PP.H2.abf PP.H3.abf PP.H4.abf 
 0.00e+00  0.00e+00  4.83e-20  1.00e+00  2.60e-10 
[1] "PP abf for shared variant: 2.6e-08%"
PP.H0.abf PP.H1.abf PP.H2.abf PP.H3.abf PP.H4.abf 
  0.00000   0.00000   0.64800   0.34500   0.00691 
[1] "PP abf for shared variant: 0.691%"
PP.H0.abf PP.H1.abf PP.H2.abf PP.H3.abf PP.H4.abf 
 0.00e+00  0.00e+00  4.83e-20  1.00e+00  2.11e-07 
[1] "PP abf for shared variant: 2.11e-05%

In [23]:
##metasoft

In [24]:
Metasoft_results <-  read.table("../../ANALYSIS/META/METASOFT/SECOND_PASS/EUROPEAN_METASOFT_SECONDPASS_RESULTS.TAB",fill=TRUE,row.names=NULL,header=FALSE)
META_TWO <- read.table(gzfile("../data/EUROPEAN_GWAS_ALLELECORRECTED.TAB"),header=T,fill=TRUE,row.names=NULL)
colnames(Metasoft_results) <- c("MARKERNAME","N","PVALUE_FE","BETA_FE","STD_FE","PVALUE_RE","BETA_RE","STD_RE","PVALUE_RE2","STAT1_RE2","STAT2_RE2","PVALUE_BE","I_SQUARE","Q","PVALUE_Q","TAU_SQUARE",
                                "PVALUE_ADIPOGEN","PVALUE_METSIM","NA1","NA2")
Metasoft_results <- Metasoft_results%>%filter(MARKERNAME!="RSID")
EUR_RESULTS <- merge(META_TWO,Metasoft_results,by="MARKERNAME")
EUR_RESULTS <- EUR_RESULTS%>%mutate(TOTAL_N=ceiling(ADIPO_N)+ceiling(METSIM_N))%>%mutate(MAF_AVG=(ADIPO_EAF*2*ADIPO_N+METSIM_EAF*2*METSIM_N)/(2*TOTAL_N))%>%
filter(ADIPO_EAF>0.005)%>%filter(METSIM_EAF>0.005)%>%select(-NA1,-N,-NA2)


In [25]:
df_expr <- list.files("../../ANALYSIS/COLOC_SUSIE/eur/",pattern='.txt',full.names = TRUE)
expr_df <- data.frame()
for (i in seq_along(df_expr))
{
tissue <- tail(str_split(df_expr,"/|.txt")[[i]],n=3)[1]
eq_df <- read.table(df_expr[i],header=T,sep="\t")%>%rename(MARKERNAME=SNP,eqtl_beta=b,eqtl_se=SE)%>%mutate(Tissue=tissue)
expr_df   <- rbind(expr_df ,eq_df)
}   

In [26]:
out_df_H4_temp <- merge(EUR_RESULTS,pip_df,by="MARKERNAME")%>%select(MARKERNAME,CHROMOSOME,POSITION,METSIM_EA,METSIM_NEA,PVALUE_FE,ADIPO_EAF,METSIM_EAF,TOTAL_N,eqtl_beta,eqtl_se,H4_abf,Tissue)%>%
filter(H4_abf>=0 )
out_df_H4 <- merge(out_df_H4_temp,expr_df,by=c("MARKERNAME","Tissue","eqtl_beta","eqtl_se"))
out_df_H3_temp <- merge(EUR_RESULTS,pip_df,by="MARKERNAME")%>%select(MARKERNAME,CHROMOSOME,POSITION,METSIM_EA,METSIM_NEA,PVALUE_FE,ADIPO_EAF,METSIM_EAF,TOTAL_N,eqtl_beta,eqtl_se,H3_abf,Tissue)%>%
filter(H3_abf>0.9 )
out_df_H3 <- merge(out_df_H3_temp,expr_df,by=c("MARKERNAME","Tissue","eqtl_beta","eqtl_se"))

In [27]:
write.table(out_df_H4,"../../ANALYSIS/COLOC_SUSIE/EUR_ANCESTRY_COLOC_H4.TAB",row.names=FALSE,quote=FALSE,sep="\t")
write.table(out_df_H3,"../../ANALYSIS/COLOC_SUSIE/EUR_ANCESTRY_COLOC_H3.TAB",row.names=FALSE,quote=FALSE,sep="\t")